# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He; psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770830276216                   -0.52    9.0    157ms
  2   -2.772142053028       -2.88       -1.32    1.0    140ms
  3   -2.772170397386       -4.55       -2.51    1.0    130ms
  4   -2.772170674326       -6.56       -3.29    1.0    110ms
  5   -2.772170722059       -7.32       -3.97    2.0    132ms
  6   -2.772170722947       -9.05       -4.66    1.0    118ms
  7   -2.772170723010      -10.20       -5.37    1.0    115ms
  8   -2.772170723015      -11.31       -5.73    2.0    134ms
  9   -2.772170723015      -12.43       -6.86    1.0    126ms
 10   -2.772170723015      -14.07       -7.36    2.0    172ms
 11   -2.772170723015      -13.97       -7.58    1.0    143ms
 12   -2.772170723015   +  -14.15       -8.48    1.0    135ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.7

1.773557958607659

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770702290882                   -0.53    8.0    164ms
  2   -2.772047483865       -2.87       -1.31    1.0    118ms
  3   -2.772082046398       -4.46       -2.59    1.0    107ms
  4   -2.772083412374       -5.86       -3.79    2.0    122ms
  5   -2.772083417225       -8.31       -4.12    2.0    123ms
  6   -2.772083417791       -9.25       -5.19    1.0    112ms
  7   -2.772083417810      -10.71       -5.65    2.0    127ms
  8   -2.772083417811      -12.34       -6.36    1.0    115ms
  9   -2.772083417811      -13.85       -7.22    1.0    126ms
 10   -2.772083417811      -14.03       -7.83    2.0    142ms
 11   -2.772083417811   +  -13.91       -8.27    1.0    118ms

Polarizability via ForwardDiff:       1.7725349592701778
Polarizability via finite difference: 1.773557958607659
